In [5]:
# Load necessary libraries
install.packages("forecast")
library(forecast)
library(nnet)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [6]:
library(forecast)

load("/content/MindSet Data.RData")

# Create a sequence of dates
start_date <- as.Date("2020-01-01")  # Replace with your desired start date
n_days <- nrow(MindSetDF)  # Assuming you want the same number of days as rows in your dataframe
date_sequence <- seq(start_date, by = "1 day", length.out = n_days)

# Add the 'Date' column to the dataframe
MindSetDF$Date <- date_sequence


Convert the data to monthly data

In [7]:
# Load necessary libraries
install.packages("dplyr")
library(dplyr)

# Convert 'Date' to Date format if it's not already
MindSetDF$Date <- as.Date(MindSetDF$Date)

# Create a new dataframe with monthly totals
MonthlyDF <- MindSetDF %>%
  group_by(YearMonth = format(Date, "%Y-%m")) %>%
  summarize(
    sum_Sales = sum(Sales),
    avg_Awareness = mean(Awareness),
    avg_Liking = mean(Liking),
    avg_Consideration = mean(Consideration),
    sum_InstagramAds = sum(InstagramAds),
    sum_TikTokAds = sum(TikTokAds),
    sum_SEA = sum(SEA),
    sum_PoSPromotions = sum(PoSPromotions),
    sum_InfluencerColabs = sum(InfluencerColabs),
    # Add additional columns as needed
    .groups = 'drop'
  )

# View the transformed dataframe
head(MonthlyDF)

# Remove the last row
MonthlyDF <- MonthlyDF[-nrow(MonthlyDF), ]

tail(MonthlyDF)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




YearMonth,sum_Sales,avg_Awareness,avg_Liking,avg_Consideration,sum_InstagramAds,sum_TikTokAds,sum_SEA,sum_PoSPromotions,sum_InfluencerColabs
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020-01,2100612,21.93346,2.7449211,5.370080,19387,15996,39158,4851,23825
2020-02,1962912,21.98035,1.4436107,4.419349,14122,10794,49295,5440,21991
2020-03,2066390,21.89729,-0.5830044,2.179261,20926,14762,37627,4702,23332
2020-04,2021128,21.92641,1.1254148,3.911787,22833,15761,38257,5266,24138
2020-05,2087244,21.95897,1.3551156,4.214358,13873,14995,47937,5147,24024
2020-06,2041290,21.98023,1.5977019,4.413806,19231,16775,46637,5217,21940


YearMonth,sum_Sales,avg_Awareness,avg_Liking,avg_Consideration,sum_InstagramAds,sum_TikTokAds,sum_SEA,sum_PoSPromotions,sum_InfluencerColabs
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022-07,2073764,21.95444,0.4612027,3.198517,21161,14286,43572,5314,25747
2022-08,2094534,21.97965,0.7166057,3.403650,22354,14397,34441,5055,26071
2022-09,1992488,21.94845,-1.1289522,1.788574,17033,14227,44174,5199,24570
2022-10,2090824,21.94612,1.7473197,4.552916,17820,14266,39959,4671,24303
2022-11,2011004,21.96213,0.9307170,3.692459,14994,11423,48433,4331,24239
2022-12,2110359,21.95885,2.1049330,4.987923,21457,13778,51129,5049,24389


In [8]:
# Load the lm model
lm_model <- readRDS("lm_model.rds")

# Load the arima models
consideration_model <- readRDS("consideration_model.rds")
liking_model <- readRDS("liking_model.rds")
awareness_model <- readRDS("awareness_model.rds")

# Data
data <- MonthlyDF

# Choose budget

sum_InstagramAds <- c(21457) # user-defined
sum_TikTokAds <- c(1300) # user-defined
sum_SEA <- c(51129) # user-defined
sum_PoSPromotions <- c(5049) # user-defined
sum_InfluencerColabs <- c(24389) # user-defined

# Simulation horizon

h <- 1 # 3 month fixed


# Forecast consideration, liking, and awareness for h periods with ARIMA models
forecast_consideration <- forecast(consideration_model, h = h, level = c(80, 95))
forecast_liking <- forecast(liking_model, h = h, level = c(80, 95))
forecast_awareness <- forecast(awareness_model, h = h, level = c(80, 95))

# Combine the forecasted values into the budget matrix
budget_matrix <- cbind(
  forecast_consideration$mean,
  forecast_liking$mean,
  forecast_awareness$mean,
  sum_InstagramAds,
  sum_TikTokAds,
  sum_SEA,
  sum_PoSPromotions,
  sum_InfluencerColabs
)

# Predict the value of Sales based on the projected budget for h periods using the lm model
#out = data.frame()
#out$probs  = predict(object = prob.model, newdata = newdata, type = "probs")
predicted_sales = predict(lm_model, newdata = data.frame(budget_matrix), interval = "prediction")
#out$score  = out$probs * out$amount

predicted_sales <- as.data.frame(predicted_sales)
head(predicted_sales)

,fit,lwr,upr
,<dbl>,<dbl>,<dbl>
1,1965887,1770637,2161137


In [16]:
predicted_sales$fit

[1] 1965887

In [17]:
summary(MonthlyDF)

  YearMonth           sum_Sales       avg_Awareness     avg_Liking     
 Length:36          Min.   :1879509   Min.   :21.87   Min.   :-1.1290  
 Class :character   1st Qu.:2026657   1st Qu.:21.93   1st Qu.: 0.7067  
 Mode  :character   Median :2075415   Median :21.96   Median : 1.3707  
                    Mean   :2055411   Mean   :21.96   Mean   : 1.2612  
                    3rd Qu.:2095879   3rd Qu.:21.98   3rd Qu.: 1.9933  
                    Max.   :2130589   Max.   :22.07   Max.   : 2.7449  
 avg_Consideration sum_InstagramAds sum_TikTokAds      sum_SEA     
 Min.   :1.789     Min.   :13873    Min.   :10794   Min.   :34441  
 1st Qu.:3.545     1st Qu.:18845    1st Qu.:13381   1st Qu.:39852  
 Median :4.213     Median :20406    Median :14246   Median :43522  
 Mean   :4.059     Mean   :20229    Mean   :14190   Mean   :43558  
 3rd Qu.:4.707     3rd Qu.:22369    3rd Qu.:14820   3rd Qu.:47056  
 Max.   :5.370     Max.   :24097    Max.   :17362   Max.   :61412  
 sum_PoSPromotions s